<a href="https://colab.research.google.com/github/Morethanmany2/Yog-Ai-Smart-Assistant-/blob/main/ML_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install pyserial if it's not already installed
!pip install pyserial

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 3.3 MB/s eta 0:00:00


In [ ]:
# Import the serial module
import serial

print("pyserial imported successfully.")


pyserial imported successfully.


In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
%%javascript
const div = document.createElement('div');
const btn = document.createElement('button');
btn.textContent = "🔌 Connect to ESP32 (Serial)";
btn.style = "font-size:20px; padding:10px;";
div.appendChild(btn);
document.body.appendChild(div);

btn.onclick = async () => {
  try {
    // Ask user to select COM port
    const port = await navigator.serial.requestPort();
    await port.open({ baudRate: 115200 });
    window.esp32Port = port;
    console.log("ESP32 Connected!");

    const reader = port.readable.getReader();
    const decoder = new TextDecoder();

    while (true) {
      const { value, done } = await reader.read();
      if (done) break;
      const text = decoder.decode(value);

      // Send to Python
      google.colab.kernel.invokeFunction(
        'notebook.onSerialData',
        [text],
        {}
      );
    }
  } catch (err) {
    console.error("Serial error:", err);
  }
};


<IPython.core.display.Javascript object>

In [ ]:
from queue import Queue
from google.colab import output

serial_buffer = Queue()

def on_serial_data(data):
    serial_buffer.put(data)

output.register_callback('notebook.onSerialData', on_serial_data)
print("Serial callback registered.")


Serial callback registered.


In [ ]:
import time

while True:
    if not serial_buffer.empty():
        print(serial_buffer.get(), end="")
    time.sleep(0.01)


ESP32 Chip model = ESP32-D0WD-V3 Rev 301
This chip has 2 cores
Chip ID: 3363472
ESP32 Chip model = ESP32-D0WD-V3 Rev 301
This chip has 2 cores
Chip ID: 3363472
ESP32 Chip model = ESP32-D0WD-V3 Rev 301
This chip has 2 cores
Chip ID: 3363472
ESP32 Chip model = ESP32-D0WD-V3 Rev 301
This chip has 2 cores
Chip ID: 3363472
ESP32 Chip model = ESP32-D0WD-V3 Rev 301
This chip has 2 cores
Chip ID: 3363472
ESP32 Chip model = ESP32-D0WD-V3 Rev 301
This chip has 2 cores
Chip ID: 3363472
ESP32 Chip model = ESP32-D0WD-V3 Rev 301
This chip has 2 cores
Chip ID: 3363472
ESP32 Chip model = ESP32-D0WD-V3 Rev 301
This chip has 2 cores
Chip ID: 3363472
ESP32 Chip model = ESP32-D0WD-V3 Rev 301
This chip has 2 cores
Chip ID: 3363472
ESP32 Chip model = ESP32-D0WD-V3 Rev 301
This chip has 2 cores
Chip ID: 3363472


KeyboardInterrupt: 

In [ ]:
H, W = 8, 8   # Your matrix size


In [ ]:
import numpy as np
def get_frame_matrix():
  if serial_buffer.empty():
    return None

  line = serial_buffer.get().strip()
  parts = line.split(",")

  if len(parts) != H * W:
    return None

  arr = np.array(parts, dtype=float).reshape(H, W)
  arr = arr/(arr.max() + 1e-8)
  return arr

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time

while True:
  frame = get_frame_matrix()
  if frame is None:
    continue

  clear_output(wait=True)
  plt.imshow(frame, cmap="jet", vmin=0, vmax=1)
  plt.colorbar()
  plt.title("Live 8x8 Pressure Matrix")
  plt.show()
  time.sleep(0.05)

KeyboardInterrupt: 

In [ ]:
def extract_features(grid):
    g = grid.astype(float)
    total_p = g.sum()
    if total_p == 0:
        return np.zeros(8)

    h, w = g.shape
    xs = np.arange(w)
    ys = np.arange(h)

    x_cop = (g.sum(axis=0) * xs).sum() / total_p
    y_cop = (g.sum(axis=1) * ys).sum() / total_p

    mid_h, mid_w = h//2, w//2
    q1 = g[:mid_h, :mid_w].sum()
    q2 = g[:mid_h, mid_w:].sum()
    q3 = g[mid_h:, :mid_w].sum()
    q4 = g[mid_h:, mid_w:].sum()

    max_p = g.max()

    return np.array([total_p, x_cop, y_cop, q1, q2, q3, q4, max_p])


In [ ]:
import pandas as pd
import numpy as np

def collect_pose(label, n=200):
    rows = []
    print(f"\nCollecting {n} samples for pose: {label}")

    while len(rows) < n:
        frame = get_frame_matrix()
        if frame is None:
            continue

        feats = extract_features(frame)
        rows.append(list(feats) + [label])  # store features + label

        if len(rows) % 20 == 0:
            print(f"{len(rows)} samples collected")

    # Create a clean DataFrame with named columns
    columns = [f"f{i}" for i in range(len(rows[0]) - 1)] + ["label"]
    df = pd.DataFrame(rows, columns=columns)
    print(f"Completed: {label}")

    return df


In [ ]:
from google.colab import files
files.download("yoga_dataset_8x8.csv")


FileNotFoundError: Cannot find file: yoga_dataset_8x8.csv

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

df = dataset.copy()
X = df.iloc[:, :-1].astype(float).values
y = df.iloc[:, -1].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y
)

clf = RandomForestClassifier(n_estimators=300)
clf.fit(X_train, y_train)

print("Accuracy:", clf.score(X_test, y_test))


Please perform 'pose1' on your sensor now...


KeyboardInterrupt: 

In [ ]:
import time
from IPython.display import clear_output

print("Starting real-time ML detection...")

while True:
    frame = get_frame_matrix()
    if frame is None:
        continue

    feats = extract_features(frame).reshape(1, -1)
    pose = clf.predict(feats)[0]

    clear_output(wait=True)
    print("📌 Detected Pose:", pose)

    time.sleep(0.1)
